In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-2"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

# create different test functions as tensorflow functions

In [3]:
def testfunc_sin(x, a, b, c):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    c=tf.cast(c,tf.float64)
    return a*tf.sin(b*x)+c

In [4]:
def testfunc_lin(x, a, b):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    return a*x+b

In [5]:
def testfunc_exp(x, a, b):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    return tf.exp(a*x)+b

# create smeared test data 

In [6]:
x_vals = np.array(np.linspace(0,2*np.pi,50))

In [7]:
y_vals_sin = []
y_vals_lin = []
y_vals_exp = []

In [8]:
for el in x_vals:
    y_vals_sin.append(testfunc_sin(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))
    y_vals_lin.append(testfunc_lin(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))
    y_vals_exp.append(testfunc_exp(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))

# it can be used any loss function with f(y_pred, y_true) which returns a scalar

In [9]:
def loss_func(y_pred, y_true):
    return tf.reduce_mean((y_pred - y_true )**2)


# create test function with weights

In [10]:
def loss_func_w(y_pred, y_true, weights):
    return tf.reduce_mean((y_pred - y_true )**2)+tf.reduce_mean(weights)
 

In [11]:
weights = np.ones(x_vals.shape[0])

# import Fitter and test it for the different functions and normal fitting + BS fitting

In [12]:
from Fitter import curve_fit, curve_fit_BS, curve_fit_raw

In [13]:
testfunclist = [testfunc_sin, testfunc_lin, testfunc_exp]
y_vals_list = [y_vals_sin, y_vals_lin, y_vals_exp]
namelist = ["sin","lin","exp"]

In [14]:
for el in zip(testfunclist,y_vals_list,namelist):
    a,b,c = curve_fit_raw(x_vals, np.array(el[1]), el[0], loss=loss_func)
    a,b,c = curve_fit_raw(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)
    a,c = curve_fit(x_vals, np.array(el[1]), el[0], loss=loss_func)
    print()
    print()
    print("function: "+el[2])
    print("parameters:")
    print(a)
    print("covariance matrix:")
    print(c)
    a,c = curve_fit(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)
    a,c = curve_fit_BS(x_vals, np.array(el[1]), el[0], loss=loss_func)
    print("parameters BS:")
    print(a)
    print("errors BS:")
    print(c)
    
    a,c = curve_fit_BS(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)




function: sin
parameters:
[0.43485382, 0.5187291, 0.50827384]
covariance matrix:
[[ 0.3318233  -0.05272642 -0.23140585]
 [-0.05272643  0.03274106  0.05202427]
 [-0.23140585  0.05202426  0.19936386]]
parameters BS:
[0.41711864 0.49374956 0.5203141 ]
errors BS:
[0.07030793 0.02043811 0.04877174]


function: lin
parameters:
[0.5135859, 0.51172286]
covariance matrix:
[[ 0.03057326 -0.09604875]
 [-0.09604874  0.4064334 ]]
parameters BS:
[0.5027779  0.52813494]
errors BS:
[0.03081433 0.04738287]


function: exp
parameters:
[0.5409596, 0.14453715]
covariance matrix:
[[ 1.8485358e-02 -7.5260842e-01]
 [-7.5260842e-01  8.0384125e+01]]
parameters BS:
[0.5682466  0.24082084]
errors BS:
[0.02698701 0.21069564]


# initial parameters should be provided as tf.Variable

In [15]:
initial_parameters = [tf.Variable(1.),tf.Variable(1.),tf.Variable(1.)]

In [16]:
a,c = curve_fit(x_vals, np.array(y_vals_sin), testfunc_sin, initial_parameters=initial_parameters)

# use Keras loss

In [17]:
lhuber = tf.keras.losses.Huber()

In [18]:
a,c = curve_fit(x_vals, np.array(y_vals_sin), testfunc_sin, loss = lhuber, initial_parameters=initial_parameters)

In [19]:
a

[0.43385446, 0.5200182, 0.5071193]